# 02 - Prompt Template

深入学习 LangChain 的提示词模板功能，学习如何创建动态和结构化的提示词。

## 学习目标
- 掌握 ChatPromptTemplate 的使用
- 学习动态变量替换
- 理解 System 和 Human 消息的区别
- 创建复杂的提示词模板

In [1]:
# 导入必要的库
import os
from dotenv import load_dotenv
from pydantic import SecretStr

# 加载环境变量
load_dotenv(override=True)

# 检查 API 密钥
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")

# 导入 LangChain 组件
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

print("✓ 环境和组件导入完成")

✓ 环境和组件导入完成


In [2]:
# 从环境变量读取配置
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")
model_name = os.getenv("MODEL_NAME", "gpt-3.5-turbo")

# 初始化模型
llm = ChatOpenAI(
    model=model_name,
    temperature=0.7,
    api_key=SecretStr(api_key),
    base_url=base_url,
)

print("✓ 模型初始化完成")

✓ 模型初始化完成


## 1. 基础提示词模板

In [3]:
# 创建简单的提示词模板
simple_template = ChatPromptTemplate.from_template("""
你是一个{role}。
请回答：{question}
""")

print("✓ 简单模板创建完成")
print(simple_template)

✓ 简单模板创建完成
input_variables=['question', 'role'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'role'], input_types={}, partial_variables={}, template='\n你是一个{role}。\n请回答：{question}\n'), additional_kwargs={})]


In [4]:
# 使用简单模板
response = llm.invoke(
    simple_template.format_messages(
        role="Python 程序员",
        question="Python 的优势是什么？"
    )
)

print("问题：Python 的优势是什么？（作为 Python 程序员回答）")
print(f"回答：{response.content}")

问题：Python 的优势是什么？（作为 Python 程序员回答）
回答：Python 作为一门广泛使用的编程语言，具有以下主要优势：

1. **语法简洁清晰**  
   Python 采用接近自然语言的语法，代码可读性强，缩进强制规范代码结构，降低了学习和维护成本。

2. **开发效率高**  
   丰富的内置功能和标准库让开发者能用更少的代码完成更多任务，快速实现原型和应用开发。

3. **跨平台兼容性好**  
   Python 可在 Windows、macOS、Linux 等多种操作系统上运行，只需编写一次代码，几乎无需修改即可跨平台部署。

4. **庞大的生态系统和第三方库**  
   PyPI（Python Package Index）提供了数十万个第三方库，覆盖 Web 开发（如 Django、Flask）、数据科学（如 NumPy、Pandas、Scikit-learn）、人工智能（如 TensorFlow、PyTorch）、自动化脚本等众多领域。

5. **强大的社区支持**  
   拥有活跃的全球开发者社区，文档齐全，教程丰富，遇到问题容易找到解决方案或获得帮助。

6. **多范式支持**  
   支持面向对象、函数式、过程式等多种编程范式，灵活适应不同项目需求。

7. **广泛应用场景**  
   从 Web 后端、数据分析、机器学习、自动化运维到科学计算、网络爬虫、游戏开发等，Python 几乎无处不在。

8. **良好的可扩展性和集成能力**  
   可轻松与 C/C++、Java、.NET 等语言集成，也支持调用系统命令和外部工具，适合构建混合系统。

9. **教育和科研友好**  
   因其易学易用，被广泛用于教学和科研领域，是许多初学者入门编程的首选语言。

这些优势使 Python 成为当今最受欢迎和最具生产力的编程语言之一。


## 2. System + Human 消息模板

In [5]:
# 创建结构化的提示词模板
system_template = SystemMessagePromptTemplate.from_template("""
你是一个专业的{role}。
你的任务是：{task}
请用{language}回答，保持{tone}的语调。
""")

human_template = HumanMessagePromptTemplate.from_template("""
用户问题：{question}
""")

# 组合成完整的提示词模板
chat_template = ChatPromptTemplate.from_messages([
    system_template,
    human_template
])

print("✓ 结构化模板创建完成")
print(chat_template)

✓ 结构化模板创建完成
input_variables=['language', 'question', 'role', 'task', 'tone'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'role', 'task', 'tone'], input_types={}, partial_variables={}, template='\n你是一个专业的{role}。\n你的任务是：{task}\n请用{language}回答，保持{tone}的语调。\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n用户问题：{question}\n'), additional_kwargs={})]


In [6]:
# 使用结构化模板
messages = chat_template.format_messages(
    role="数据科学家",
    task="解释机器学习概念",
    language="中文",
    tone="专业且易懂",
    question="什么是过拟合？",
)

response = llm.invoke(messages)
print("问题：什么是过拟合？")
print(f"回答：{response.content}")

问题：什么是过拟合？
回答：过拟合（Overfitting）是机器学习中一个常见问题，指的是模型在训练数据上表现非常好（例如，训练误差很低），但在未见过的测试数据或真实数据上表现明显变差的现象。

通俗来说，过拟合意味着模型“死记硬背”了训练数据中的细节和噪声，而不是学习到数据背后的通用规律。这导致它缺乏泛化能力（generalization ability），即无法很好地应对新样本。

### 举个例子：
假设你正在教一个学生识别猫的图片。如果这个学生只记住训练集中每一张猫图的具体像素，而没有理解“猫”的本质特征（比如有胡须、尖耳朵等），那么当他看到一张新的猫图片时，可能就无法正确识别——这就是过拟合。

### 过拟合的典型表现：
- 训练准确率很高（如 99%），但验证/测试准确率明显偏低（如 70%）。
- 模型复杂度高（例如深度很深的神经网络、高阶多项式回归等）。
- 训练数据量相对较少，不足以支撑模型复杂度。

### 如何缓解过拟合？
1. **增加训练数据**：更多样化的数据有助于模型学习更通用的模式。
2. **简化模型**：降低模型复杂度（如减少神经网络层数、限制决策树深度）。
3. **正则化（Regularization）**：如 L1/L2 正则，在损失函数中加入惩罚项，抑制模型参数过大。
4. **交叉验证（Cross-validation）**：用于更可靠地评估模型性能。
5. **早停法（Early stopping）**：在训练过程中监控验证误差，当其不再下降时提前停止训练。
6. **Dropout（用于神经网络）**：在训练时随机“关闭”一部分神经元，防止对特定特征过度依赖。

总之，过拟合是模型“学得太好反而不会举一反三”的问题，解决的关键在于提升模型的泛化能力。


## 3. 多模板对比

In [7]:
# 创建不同角色的模板
roles = [
    {"role": "幼儿园老师", "tone": "耐心温柔", "language": "简单的中文"},
    {"role": "大学教授", "tone": "学术严谨", "language": "专业的中文"},
    {"role": "脱口秀演员", "tone": "幽默风趣", "language": "轻松的中文"}
]

question = "为什么天空是蓝色的？"

for role_info in roles:
    print(f"\n--- {role_info['role']}的回答 ---")
    
    messages = chat_template.format_messages(
        role=role_info["role"],
        task="解释自然现象",
        language=role_info["language"],
        tone=role_info["tone"],
        question=question
    )
    
    response = llm.invoke(messages)
    print(response.content)


--- 幼儿园老师的回答 ---
小朋友，你问得真棒！✨

天空看起来是蓝色的，是因为阳光照到地球的时候，会遇到空气中的小分子。阳光看起来是白色的，但其实它里面藏着很多种颜色，就像彩虹一样。

这些颜色中，蓝色的光波比较短，很容易被空气里的小分子“弹来弹去”——我们叫这“散射”。因为蓝色光被散射得到处都是，所以当我们抬头看天空，就看到一片蓝蓝的啦！

不过呀，在日出或日落的时候，太阳在低低的地方，光要穿过更厚的空气，这时候蓝色光被“弹走”了，剩下的红红、橙橙的光就更明显，所以天空会变成漂亮的红色或橙色哦！

是不是很神奇？大自然就像一位会变魔术的画家呢！🎨💙

--- 大学教授的回答 ---
天空呈现蓝色，主要是由于大气对太阳光的瑞利散射（Rayleigh scattering）效应所致。

太阳辐射出的白光实际上是由不同波长的电磁波组成的连续光谱，其中可见光波段大致涵盖400纳米（紫光）至700纳米（红光）。当太阳光进入地球大气层时，会与大气中的气体分子（如氮气和氧气）以及微小颗粒发生相互作用。根据瑞利散射理论，散射强度 \( I \) 与入射光波长 \( \lambda \) 的四次方成反比，即：

\[
I \propto \frac{1}{\lambda^4}
\]

这意味着波长较短的蓝光（约450–495 nm）比波长较长的红光（约620–750 nm）更容易被散射。具体而言，蓝光的散射强度约为红光的 \( (700/450)^4 \approx 6 \) 倍。因此，在白天，当阳光穿过大气层时，蓝光在各个方向上被大量散射，使得无论我们朝哪个方向观察（除直视太阳外），都会接收到大量来自大气中散射的蓝光，从而感知天空为蓝色。

值得注意的是，尽管紫光波长更短、理论上应被散射得更强，但人眼对紫光的敏感度较低，且太阳光谱中紫光成分相对较少，加之部分紫外辐射被臭氧层吸收，因此我们看到的天空主要呈现蓝色而非紫色。

综上所述，天空之所以呈蓝色，是瑞利散射机制、太阳光谱分布以及人眼视觉响应三者共同作用的结果。

--- 脱口秀演员的回答 ---
哎哟，这个问题问得好啊！你有没有小时候仰头看天，突然冒出一个哲学问题：“为啥天不是绿色的？不然我抬头就能吃抹茶冰淇淋了！”  

其实啊，天空是蓝色的，全靠太阳光在“搞事情”。太阳光看起来是白的，但其实它是个“彩虹批发商”—

## 4. 条件模板

In [8]:
# 创建带条件的模板
def create_conditional_template(difficulty):
    if difficulty == "初级":
        return ChatPromptTemplate.from_template("""
        你是一个耐心的初学者教练。
        请用最简单的语言解释：{topic}
        避免使用专业术语。
        """)
    elif difficulty == "中级":
        return ChatPromptTemplate.from_template("""
        你是一个经验丰富的讲师。
        请解释：{topic}
        可以适当使用专业术语，但要解释清楚。
        """)
    elif difficulty == "高级":
        return ChatPromptTemplate.from_template("""
        你是该领域的专家。
        请深入分析：{topic}
        可以使用高级概念和技术细节。
        """)
    else:
        return ChatPromptTemplate.from_template("请解释：{topic}")

# 测试不同难度
topic = "神经网络"
difficulties = ["初级", "中级", "高级"]

for diff in difficulties:
    print(f"\n=== {diff}解释 ===")
    template = create_conditional_template(diff)
    
    messages = template.format_messages(topic=topic)
    response = llm.invoke(messages)
    print(response.content)


=== 初级解释 ===
当然可以！我们用一个简单的比喻来理解神经网络。

想象你教一个小孩子认猫。一开始，他不知道什么是猫。于是你给他看很多图片，有的是猫，有的不是。每次他猜对了，你就表扬他；猜错了，你就轻轻纠正他。

慢慢地，他开始注意到一些规律：猫有尖耳朵、胡须、圆眼睛……虽然他不能一下子说出所有特征，但他越来越会判断“这看起来像不像猫”。

神经网络就像这个学习中的小孩。它是一套“聪明的规则”，通过看大量例子（比如成千上万张猫和狗的照片），自己慢慢学会找出其中的规律。它不会一开始就懂，但经过反复练习，就能越来越准地完成任务，比如识别图片、翻译句子，或者预测天气。

简单说：  
**神经网络 = 一个通过大量例子自己学习怎么做事的“电子大脑”**。

它不靠人一条条写规则，而是从数据中“摸索”出答案——就像我们人类通过经验学习一样。

=== 中级解释 ===
当然可以！作为一名经验丰富的讲师，我很高兴为你清晰、系统地解释“神经网络”这个概念。

---

### 什么是神经网络？

**神经网络**（Neural Network）是一种受生物大脑结构启发的计算模型，是**人工智能**（AI）和**机器学习**（Machine Learning）中的核心工具之一。它的基本思想是：通过大量简单的处理单元（称为“神经元”）相互连接，模拟人脑处理信息的方式，从而从数据中自动学习规律。

---

### 1. 基本组成单位：人工神经元

神经网络的基本单元是**人工神经元**（Artificial Neuron），也叫**节点**（Node）或**单元**（Unit）。

一个神经元接收多个输入（比如来自其他神经元的信号），对这些输入进行加权求和，再加上一个偏置（bias），然后通过一个**激活函数**（Activation Function）产生输出。

数学表达如下：

\[
z = \sum_{i=1}^{n} w_i x_i + b
\]
\[
a = f(z)
\]

- \(x_i\)：输入值（例如特征）
- \(w_i\)：对应的权重（weight），表示该输入的重要性
- \(b\)：偏置（bias），用于调整输出的基线
- \(f(\cdot)\)：激活函数（如 ReLU、Sigmoid、Tanh 等），引入非线性，使网络能拟合复杂函数
- \

## 5. 模板练习

In [10]:
# 创建一个代码生成模板
code_template = ChatPromptTemplate.from_template("""
你是一个专业的{language}程序员。
请根据以下要求生成代码：

功能：{functionality}
语言：{language}
框架：{framework}
特殊要求：{requirements}

请提供完整的代码示例，并添加必要的注释。
""")

# 生成一个 Python Flask API
messages = code_template.format_messages(
    language="Python",
    functionality="创建一个简单的 REST API",
    framework="Flask",
    requirements="包含 GET 和 POST 接口，返回 JSON 格式数据"
)

response = llm.invoke(messages)
print("生成的 Flask API 代码：")
print(response.content)

生成的 Flask API 代码：
```python
"""
简单的 REST API 示例
使用 Flask 框架实现 GET 和 POST 接口，返回 JSON 格式数据
"""

from flask import Flask, jsonify, request

# 创建 Flask 应用实例
app = Flask(__name__)

# 模拟数据库 - 存储用户数据
users = [
    {"id": 1, "name": "张三", "email": "zhangsan@example.com"},
    {"id": 2, "name": "李四", "email": "lisi@example.com"}
]

# 获取所有用户的 GET 接口
@app.route('/api/users', methods=['GET'])
def get_users():
    """
    获取所有用户信息
    URL: /api/users
    方法: GET
    返回: JSON 格式的用户列表
    """
    return jsonify({
        "success": True,
        "data": users,
        "message": "获取用户列表成功"
    })

# 根据 ID 获取单个用户的 GET 接口
@app.route('/api/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    """
    根据用户ID获取单个用户信息
    URL: /api/users/<user_id>
    方法: GET
    参数: user_id (路径参数)
    返回: JSON 格式的用户信息或错误信息
    """
    # 查找用户
    user = next((user for user in users if user["id"] == user_id), None)
    
    if user:
        return jsonify({
            "success": True,
            "data": user,


## 总结

在这个示例中，我们学习了：

1. **基础模板**: 使用 `from_template()` 创建简单模板
2. **结构化模板**: 分离 System 和 Human 消息
3. **动态变量**: 使用 `{variable}` 占位符
4. **条件模板**: 根据参数动态创建不同的模板
5. **复杂应用**: 代码生成、多角色对话等

### 最佳实践
- System 消息设定 AI 的角色和行为准则
- Human 消息包含具体的用户请求
- 使用有意义的变量名
- 根据使用场景调整模板复杂度

### 下一步
尝试创建自己的提示词模板，用于特定的应用场景！